In [ ]:
!pip install sentence-transformers

import random
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import torch


nltk.download('punkt')
sent_embedding_model = "all-MiniLM-L6-v2"
device_used = "cuda"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
random_seed = 70
random.seed(random_seed)

torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [ ]:
def split_sentence(paragraph):
    sentences = nltk.sent_tokenize(paragraph)
    return sentences

def sent_embedding(paragraph):
    sentences = split_sentence(paragraph)
    if torch.cuda.is_available():
        model = SentenceTransformer(sent_embedding_model, device=device_used)
    else:
        model = SentenceTransformer(sent_embedding_model)
    sentence_embeddings = model.encode(sentences)
    return sentence_embeddings

In [ ]:
embeddings = sent_embedding("Machine learning is part of data science. A movie theater, cinema, or cinema hall, also known as a movie house, picture house, picture theater or simply theater. Data science is part of the curriculum in many courses. The film is projected with a movie projector onto a large projection screen at the front of the auditorium while the dialogue, sounds and music are played through speakers. Movie theatres stand in a long tradition of theaters that could house all kinds of entertainment. we are doing NLP projeckt")

In [ ]:
embeddings.shape

(6, 384)

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import numpy as np
from scipy.spatial.distance import cdist

def elbow(embed, means):
    score = cdist(embed, means)
    score = np.sum(np.min(score, axis=1))/embed.shape[0]
    return score

def silhouette(embed, classes):
    scores = silhouette_score(embed, classes, metric="euclidean")
    return scores

#GMM clustering
def clustering(sentence_embeddings):
    cluster_count = 2
    scores = []
    labels_array = []
    prob_array = []
    while(cluster_count<5):
        gmm = GaussianMixture(cluster_count)
        labels = gmm.fit_predict(sentence_embeddings) # sentence_embedding shape -> (num_samples, num_features)
        prob_array.append(gmm.predict_proba(sentence_embeddings))
        labels_array.append(labels)
        scores.append(silhouette(sentence_embeddings, labels))
        # print(scores)
        #scores.append(elbow(sentence_embeddings, gmm.means_))
        cluster_count+=1

    least = np.argmax(scores)

    return labels_array[least], prob_array

labels_arr, prob_arr = clustering(embeddings)

print(labels_arr)
print()
print(prob_arr)

[2 1 2 1 1 0]

[array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.]]), array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])]
